---
## Setup and Imports

In [1]:
import os
import sys
from pathlib import Path

# Add project root to path (not 'src' directory)
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Core imports
from src.core.models import LoopConfig, FinalResult
from src.loop.orchestrator import LoopOrchestrator
from src.prompts.builder import MSAPromptBuilder, HARPromptBuilder
from src.llm.factory import ProviderFactory
from src.simlp.client import SimLPClient

# Utilities
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✓ All imports successful!")
print(f"Working directory: {Path.cwd()}")
print(f"Project root: {project_root}")

✓ All imports successful!
Working directory: /Users/gphome/Desktop/projects/thesis-ds/feedback-loop/notebooks
Project root: /Users/gphome/Desktop/projects/thesis-ds/feedback-loop


---
## 1. Basic Setup

Let's configure the three main components:
1. **Prompt Builder** - Constructs prompts for the LLM
2. **LLM Provider** - Generates RTEC rules
3. **SimLP Client** - Evaluates rule quality

In [2]:
# 1. Configure Loop Settings
config = LoopConfig(
    provider="openai",
    objective="Generate accurate RTEC rules for activity recognition",
    max_iterations=1,
    convergence_threshold=0.9,
    batch_size=1,
    retry_limit=3
)

print("Loop Configuration:")
print(f"  Max Iterations: {config.max_iterations}")
print(f"  Convergence Threshold: {config.convergence_threshold}")
print(f"  Provider: {config.provider}")

Loop Configuration:
  Max Iterations: 1
  Convergence Threshold: 0.9
  Provider: openai


In [3]:
# 2. Create Prompt Builder (MSA domain example)
prompt_builder = MSAPromptBuilder()

print("✓ Prompt builder created for MSA domain")
print(f"  Domain-specific templates loaded")

✓ Prompt builder created for MSA domain
  Domain-specific templates loaded


In [4]:
# 3. Create LLM Provider
# Make sure to set your API key as an environment variable
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    print("⚠️  Warning: OPENAI_API_KEY not set. Using mock provider for demo.")
    # For demo purposes, you could use a mock provider
    # In production, set the API key:
    # export OPENAI_API_KEY='your-key-here'

llm_provider = ProviderFactory.create(
    provider_name="openai",
    api_key=api_key
)

print("✓ LLM provider created")
print(f"  Provider: OpenAI")

✓ LLM provider created
  Provider: OpenAI


In [5]:
# 4. Create SimLP Client
simlp_client = SimLPClient(
    reference_rules_dir="../data/ground_truth",
    log_dir="../logs/simlp"
)

print("✓ SimLP client created")
print(f"  Reference rules directory: ../data/reference_rules")
print(f"  Logs will be saved to: ../logs/simlp")

✓ SimLP client created
  Reference rules directory: ../data/reference_rules
  Logs will be saved to: ../logs/simlp


---
## 2. Creating the Orchestrator

Now we combine all components into the orchestrator:

In [6]:
from datetime import datetime
LOG_DIR = Path(project_root / 'logs')

In [7]:
# Create the orchestrator with all components
orchestrator = LoopOrchestrator(
    prompt_builder=prompt_builder,
    llm_provider=llm_provider,
    simlp_client=simlp_client,
    config=config,
    log_dir=LOG_DIR
)

print("✓ LoopOrchestrator initialized!")
print("\nReady to generate RTEC rules with iterative refinement.")

✓ LoopOrchestrator initialized!

Ready to generate RTEC rules with iterative refinement.


In [8]:
orchestrator._logger_config.verbose

False

---
## Enhanced Logging Options

The orchestrator supports verbose logging with file output for debugging and monitoring.

In [9]:
# Option 2: Custom log file path
from src.loop.logging_config import setup_orchestrator_logging

# Create custom logger
custom_logger = setup_orchestrator_logging(
    verbose=True,
    log_file="./../logs/my_experiment.log",
    log_level="DEBUG"
)

orchestrator_custom = LoopOrchestrator(
    prompt_builder=prompt_builder,
    llm_provider=llm_provider,
    simlp_client=simlp_client,
    config=config,
    logger=custom_logger  # Use custom logger
)

print("✓ Orchestrator with custom logger created")
print(f"  Custom log file: ./logs/my_experiment.log")

✓ Orchestrator with custom logger created
  Custom log file: ./logs/my_experiment.log


In [10]:
orchestrator_custom.logger

<Logger orchestrator (DEBUG)>

In [11]:
# Option 3: Standard logging (INFO level, file only when needed)
orchestrator_standard = LoopOrchestrator(
    prompt_builder=prompt_builder,
    llm_provider=llm_provider,
    simlp_client=simlp_client,
    config=config,
    log_file="./logs/info.log"  # File logging without verbose console
)

print("✓ Orchestrator with standard logging created")
print("  Console: INFO level")
print("  File: DEBUG level at ./logs/info.log")

✓ Orchestrator with standard logging created
  Console: INFO level
  File: DEBUG level at ./logs/info.log


---
## 3. Running the Feedback Loop

Let's generate rules for the 'active' activity in the MSA domain.

In [13]:
# Run the feedback loop
domain = "MSA"
activity = "gap"

print(f"Starting feedback loop for {domain}/{activity}...\n")

result = orchestrator.run(domain=domain, activity=activity)

print("\n✓ Feedback loop completed!")

Starting feedback loop for MSA/gap...

INFO     | ================================================================================
INFO     | Starting feedback loop for MSA/gap
INFO     |   Max iterations: 1
INFO     |   Convergence threshold: 0.9
INFO     | ================================================================================
INFO     | ITERATION 1: Generating initial rules...
INFO     | Evaluating initial rules...
Concepts defined in both event descriptions: 
[('gap', 'initiatedAt'), ('gap', 'terminatedAt')]

Concepts defined only in generated event description: 
[]

Concepts defined only in ground event description: 
[]

Similarity for definition: ('gap', 'initiatedAt') is 1.0
Similarity for definition: ('gap', 'terminatedAt') is 1.0
Event Description Similarity is: 
1.0
INFO     | Iteration 1 complete: score=1.0000, converged=True
INFO     | 
INFO     | FEEDBACK LOOP COMPLETED
INFO     | ================================================================================
INF

In [14]:
print(result.states[0].completed_requests[0].content)

The activity "gap" is expressed as a simple fluent with two arguments: the "Vessel" and the "Location" (nearPorts or farFromPorts). 

The activity "gap" starts when we stop receiving messages from a vessel that is near some port. We use an "initiatedAt" rule to describe this initiation condition. The output is the fluent "gap", which has value "nearPorts". The "initiatedAt" rule includes a built-in event named "gap_start" with one argument, i.e. "Vessel". We verify that the vessel is currently near some port by requiring that the fluent-value pair “withinArea(Vessel, nearPorts)=true” must hold. This rule in the language of RTEC is: 

```prolog
initiatedAt(gap(Vessel)=nearPorts, T) :-
    happensAt(gap_start(Vessel), T),
    holdsAt(withinArea(Vessel, nearPorts)=true, T).
```

The activity "gap" may also start when a vessel that is far from all ports stops sending messages. In this case, the "gap" fluent has value "farFromPorts". We use an "initiatedAt" rule to express this initiation c

In [15]:
print(result.states[0].rules)

initiatedAt(gap(Vessel)=nearPorts, T) :-
    happensAt(gap_start(Vessel), T),
    holdsAt(withinArea(Vessel, nearPorts)=true, T).

initiatedAt(gap(Vessel)=farFromPorts, T) :-
    happensAt(gap_start(Vessel), T),
    not holdsAt(withinArea(Vessel, nearPorts)=true, T).

terminatedAt(gap(Vessel)=_Location, T) :-
    happensAt(gap_end(Vessel), T).


---
## 4. Analyzing the Results

Let's examine what the orchestrator produced.

In [16]:
# Summary statistics
print("=" * 60)
print("FEEDBACK LOOP SUMMARY")
print("=" * 60)
print(f"Converged: {result.summary['converged']}")
print(f"Iterations Used: {result.summary['iterations_used']}")
print(f"Final Score: {result.summary['final_score']:.3f}")
print(f"Best Score: {result.summary['best_score']:.3f}")
print(f"Best Iteration: {result.summary['best_iteration']}")
print(f"\nImprovement: {result.summary['improvement']:.3f}")
print(f"Improvement Rate: {result.summary['improvement_rate']:.3f} per iteration")
print(f"\nTotal Tokens Used: {result.summary['total_tokens']}")
print(f"Average Latency: {result.summary['avg_latency_ms']:.2f} ms")
print("=" * 60)

FEEDBACK LOOP SUMMARY
Converged: True
Iterations Used: 1
Final Score: 1.000
Best Score: 1.000
Best Iteration: 1

Improvement: 0.000
Improvement Rate: 0.000 per iteration

Total Tokens Used: 5197
Average Latency: 22259.33 ms


In [17]:
# Display the best rules
print("\n" + "=" * 60)
print("BEST RULES GENERATED")
print("=" * 60)
print(result.best_rules[0])
print("=" * 60)


BEST RULES GENERATED
initiatedAt(gap(Vessel)=nearPorts, T) :-
    happensAt(gap_start(Vessel), T),
    holdsAt(withinArea(Vessel, nearPorts)=true, T).

initiatedAt(gap(Vessel)=farFromPorts, T) :-
    happensAt(gap_start(Vessel), T),
    not holdsAt(withinArea(Vessel, nearPorts)=true, T).

terminatedAt(gap(Vessel)=_Location, T) :-
    happensAt(gap_end(Vessel), T).


---
## 5. Visualizing Convergence

Let's visualize how the similarity score improved over iterations.

In [ ]:
# Extract scores for plotting
iterations = list(range(1, len(result.states) + 1))
scores = [state.evaluations[0].score for state in result.states]

# Create convergence plot
plt.figure(figsize=(10, 6))
plt.plot(iterations, scores, marker='o', linewidth=2, markersize=8, label='Similarity Score')
plt.axhline(y=config.convergence_threshold, color='r', linestyle='--', 
            label=f'Threshold ({config.convergence_threshold})')
plt.xlabel('Iteration', fontsize=12)
plt.ylabel('Similarity Score', fontsize=12)
plt.title(f'Feedback Loop Convergence - {domain}/{activity}', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.legend(fontsize=10)
plt.ylim([0, 1.05])

# Annotate best score
best_idx = result.summary['best_iteration'] - 1
plt.annotate(f"Best: {result.summary['best_score']:.3f}",
             xy=(iterations[best_idx], scores[best_idx]),
             xytext=(10, 10), textcoords='offset points',
             bbox=dict(boxstyle='round,pad=0.5', fc='yellow', alpha=0.7),
             arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0'))

plt.tight_layout()
plt.show()

print(f"✓ Convergence {'achieved' if result.summary['converged'] else 'not achieved'}")

In [ ]:
# Token usage over iterations
tokens_per_iter = [state.completed_requests[0].tokens_used for state in result.states]

plt.figure(figsize=(10, 5))
plt.bar(iterations, tokens_per_iter, color='steelblue', alpha=0.7)
plt.xlabel('Iteration', fontsize=12)
plt.ylabel('Tokens Used', fontsize=12)
plt.title('Token Usage per Iteration', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3, axis='y')

# Add average line
plt.axhline(y=result.summary['avg_tokens_per_iteration'], 
            color='red', linestyle='--', linewidth=2,
            label=f"Average: {result.summary['avg_tokens_per_iteration']:.0f}")
plt.legend()

plt.tight_layout()
plt.show()

print(f"Total tokens used: {result.summary['total_tokens']}")

---
## 6. Comparing Different Activities

Let's run the feedback loop for multiple activities and compare results.

In [ ]:
# Run for multiple activities
activities = ['active', 'inactive', 'walking']
results = {}

print("Running feedback loops for multiple activities...\n")

for activity in activities:
    print(f"Processing {activity}...")
    try:
        result = orchestrator.run(domain="MSA", activity=activity)
        results[activity] = result
        print(f"  ✓ {activity}: {result.summary['iterations_used']} iterations, "
              f"score={result.summary['final_score']:.3f}\n")
    except Exception as e:
        print(f"  ✗ {activity}: Failed - {e}\n")

print(f"Completed {len(results)}/{len(activities)} activities")

In [ ]:
# Create comparison DataFrame
comparison_data = []
for activity, result in results.items():
    comparison_data.append({
        'Activity': activity,
        'Converged': result.summary['converged'],
        'Iterations': result.summary['iterations_used'],
        'Final Score': result.summary['final_score'],
        'Improvement': result.summary['improvement'],
        'Total Tokens': result.summary['total_tokens'],
        'Avg Latency (ms)': result.summary['avg_latency_ms']
    })

df_comparison = pd.DataFrame(comparison_data)
print("\n" + "=" * 80)
print("ACTIVITY COMPARISON")
print("=" * 80)
print(df_comparison.to_string(index=False))
print("=" * 80)

In [ ]:
# Visualize comparison
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Plot 1: Final scores
axes[0].bar(df_comparison['Activity'], df_comparison['Final Score'], 
            color='steelblue', alpha=0.7)
axes[0].axhline(y=config.convergence_threshold, color='r', linestyle='--', 
                label='Threshold')
axes[0].set_ylabel('Final Score')
axes[0].set_title('Final Similarity Scores')
axes[0].set_ylim([0, 1])
axes[0].legend()

# Plot 2: Iterations required
axes[1].bar(df_comparison['Activity'], df_comparison['Iterations'], 
            color='coral', alpha=0.7)
axes[1].set_ylabel('Iterations')
axes[1].set_title('Iterations Required')

# Plot 3: Token usage
axes[2].bar(df_comparison['Activity'], df_comparison['Total Tokens'], 
            color='seagreen', alpha=0.7)
axes[2].set_ylabel('Total Tokens')
axes[2].set_title('Token Usage')

plt.tight_layout()
plt.show()

---
## 7. Advanced Configuration

### Tuning Convergence Threshold

Let's see how different thresholds affect the results.

In [ ]:
# Test different thresholds
thresholds = [0.8, 0.85, 0.9, 0.95]
threshold_results = []

for threshold in thresholds:
    # Create config with different threshold
    test_config = LoopConfig(
        provider="openai",
        objective="Generate RTEC rules",
        max_iterations=5,
        convergence_threshold=threshold,
        batch_size=1,
        retry_limit=3
    )
    
    # Create new orchestrator
    test_orchestrator = LoopOrchestrator(
        prompt_builder=prompt_builder,
        llm_provider=llm_provider,
        simlp_client=simlp_client,
        config=test_config
    )
    
    # Run
    result = test_orchestrator.run(domain="MSA", activity="active")
    
    threshold_results.append({
        'Threshold': threshold,
        'Converged': result.summary['converged'],
        'Iterations': result.summary['iterations_used'],
        'Final Score': result.summary['final_score'],
        'Total Tokens': result.summary['total_tokens']
    })

df_thresholds = pd.DataFrame(threshold_results)
print("\nThreshold Impact Analysis:")
print(df_thresholds)

### Using Different Fluent Types

The prompt builder supports different example types to optimize token usage.

In [ ]:
# Compare fluent types: 'simple', 'static', 'both'
fluent_types = ['simple', 'static', 'both']

for fluent_type in fluent_types:
    # Build initial prompt with specific fluent type
    messages = prompt_builder.build_initial(activity='active', fluent_type=fluent_type)
    
    # Calculate token estimate (rough)
    total_chars = sum(len(msg['content']) for msg in messages)
    estimated_tokens = total_chars // 4  # Rough estimate: 1 token ≈ 4 chars
    
    print(f"\n{fluent_type.upper()}:")
    print(f"  Messages: {len(messages)}")
    print(f"  Total characters: {total_chars}")
    print(f"  Estimated tokens: {estimated_tokens}")

---
## 8. Inspecting Iteration States

Let's dive deep into what happened in each iteration.

In [ ]:
# Get the first result for detailed analysis
detailed_result = result  # Using the last result from above

print("DETAILED ITERATION ANALYSIS\n")
print("=" * 80)

for state in detailed_result.states:
    eval_result = state.evaluations[0]
    response = state.completed_requests[0]
    
    print(f"\n{'='*80}")
    print(f"ITERATION {state.iteration}")
    print(f"{'='*80}")
    
    print(f"\nRequest ID: {response.request_id}")
    print(f"Model: {response.model}")
    print(f"Tokens: {response.tokens_used}")
    print(f"Latency: {response.latency_ms:.2f} ms")
    print(f"Finish Reason: {response.finish_reason}")
    
    print(f"\nEvaluation:")
    print(f"  Score: {eval_result.score:.4f}")
    print(f"  Matches Reference: {eval_result.matches_reference}")
    print(f"  Converged: {state.converged}")
    
    if eval_result.feedback:
        print(f"\nFeedback:")
        print(f"  {eval_result.feedback}")
    
    if eval_result.issues:
        print(f"\nIssues Identified:")
        for issue in eval_result.issues:
            print(f"  • {issue}")
    
    print(f"\nGenerated Rules (first 200 chars):")
    print(f"  {response.content[:200]}...")

---
## 9. Error Handling and Edge Cases

In [ ]:
# Example: Handling missing reference rules
try:
    result = orchestrator.run(domain="MSA", activity="nonexistent_activity")
except Exception as e:
    print(f"Expected error caught: {type(e).__name__}")
    print(f"Message: {str(e)}")
    print("\n✓ Error handling working correctly")

In [ ]:
# Example: Maximum iterations reached without convergence
strict_config = LoopConfig(
    provider="openai",
    objective="Generate RTEC rules",
    max_iterations=2,  # Very low
    convergence_threshold=0.99,  # Very high
    batch_size=1,
    retry_limit=3
)

strict_orchestrator = LoopOrchestrator(
    prompt_builder=prompt_builder,
    llm_provider=llm_provider,
    simlp_client=simlp_client,
    config=strict_config
)

result = strict_orchestrator.run(domain="MSA", activity="active")

print("Results with strict config:")
print(f"  Converged: {result.summary['converged']}")
print(f"  Iterations: {result.summary['iterations_used']}")
print(f"  Final Score: {result.summary['final_score']:.3f}")
print(f"  Notes: {result.notes}")
print("\n✓ Orchestrator handles non-convergence gracefully")

---
## 10. Saving and Loading Results

In [ ]:
import json

# Save result to JSON
output_dir = Path("../output")
output_dir.mkdir(exist_ok=True)

output_file = output_dir / f"result_{domain}_{activity}.json"

# Convert to dict (Pydantic model)
result_dict = result.model_dump()

with open(output_file, 'w') as f:
    json.dump(result_dict, f, indent=2, default=str)

print(f"✓ Result saved to: {output_file}")
print(f"  File size: {output_file.stat().st_size} bytes")

In [ ]:
# Save best rules to a Prolog file
rules_file = output_dir / f"rules_{domain}_{activity}.pl"

with open(rules_file, 'w') as f:
    f.write(f"% RTEC Rules for {domain}/{activity}\n")
    f.write(f"% Generated by LoopOrchestrator\n")
    f.write(f"% Iterations: {result.summary['iterations_used']}\n")
    f.write(f"% Final Score: {result.summary['final_score']:.3f}\n\n")
    f.write(result.best_rules[0])

print(f"✓ Rules saved to: {rules_file}")

---
## Summary and Key Takeaways

### What We Learned:

1. **Setup**: Creating the orchestrator requires three components:
   - Prompt builder (domain-specific)
   - LLM provider (via factory)
   - SimLP client (for evaluation)

2. **Configuration**: Key parameters:
   - `max_iterations`: Trade-off between quality and cost
   - `convergence_threshold`: Balance between precision and feasibility
   - Provider settings: Model selection, temperature, etc.

3. **Results Analysis**:
   - Convergence patterns vary by activity
   - Token usage typically increases with refinement
   - Best rules aren't always from the last iteration

4. **Best Practices**:
   - Start with moderate thresholds (0.85-0.9)
   - Use 3-5 max iterations for initial experiments
   - Monitor token usage for cost control
   - Save results for reproducibility

### Next Steps:

- **Scale up**: Process multiple activities in batch
- **Experiment**: Try different LLM models and prompts
- **Analyze**: Study convergence patterns across domains
- **Optimize**: Fine-tune thresholds based on results
- **Integrate**: Use in your thesis evaluation pipeline